In [1]:
import os 
import sys 
import json 
from pathlib import Path 
from functools import cache
from itertools import product

cur_path = os.path.abspath(".")
if cur_path not in sys.path: 
    sys.path.append(cur_path)

import numpy as np 
import pandas as pd 
import altair as alt 
from altair import datum
from subgrounds.subgrounds import Subgrounds, Subgraph
from subgrounds.pagination import ShallowStrategy

from utils import ddf, remove_prefix, load_subgraph, remove_keys

In [2]:
sg, bs = load_subgraph()

## Siloed Asset Breakdown 

In [3]:
from collections import OrderedDict

# See UI logic in this directory for reference 
# https://github.com/BeanstalkFarms/Beanstalk-UI/tree/master/src/components/Analytics/Silo
beanstalk_addr = '0xC1E088fC1323b20BCBee9bd1B9fC9546db5624C5'
token_addrs = OrderedDict(dict(
    bean='0xBEA0000029AD1c77D3d5D23Ba2D8893dB9d1Efab'.lower(), 
    bean_3crv='0xc9C32cd16Bf7eFB85Ff14e0c8603cc90F6F2eE49'.lower(), 
    ur_bean='0x1BEA0050E63e05FBb5D8BA2f10cf5800B6224449'.lower(), 
    ur_bean_3crv='0x1BEA3CcD22F4EBd3d37d731BA31Eeca95713716D'.lower(), 
))
asset_ids = [f"{beanstalk_addr.lower()}-{token_addr.lower()}" for token_addr in token_addrs.values()]
queries = [
    bs.Query.siloAssetHourlySnapshots(
        first=10000, orderBy="season", orderDirection="asc",
        where={"season_gte": 6074, "siloAsset": aid}
    )
    for aid in asset_ids 
]
df_snaps_raw = pd.concat([
    sg.query_df(
        [
            q.id, 
            q.season, 
            q.timestamp, 
            q.totalDepositedAmount, 
            q.totalDepositedBDV, 
        ],
        pagination_strategy=ShallowStrategy
    )
    for q in queries
])

In [4]:

from collections import defaultdict

df_dep = df_snaps_raw.copy()
df_dep = remove_prefix(df_dep, 'siloAssetHourlySnapshots_').sort_values('timestamp').reset_index(drop=True)
df_dep['token'] = df_dep['id'].apply(lambda v: v.split('-')[1])
# df_dep = df_dep.groupby(['season', 'token']).last().reset_index()
# seasons = sorted(df_dep.season.unique())
# tokens = df_dep.token.unique()
# records = []
token_addr_inv = {v: k for k, v in token_addrs.items()}
# token_mult = OrderedDict(dict(
#     bean=1e-6, 
#     bean_3crv=1e-18, 
#     ur_bean=1e-6, 
#     ur_bean_3crv=1e-12, 
# ))
# last_value = defaultdict(float)
# for s, t in product(seasons, tokens):
#     metrics = {}
#     for m in ['totalDepositedAmount', 'totalDepositedBDV']: 
#         v = df_dep.loc[(df_dep.season == s) & (df_dep.token == t)][m]
#         if len(v): 
#             v = v.values[0]
#         else: 
#             v = last_value.get((t,m), 0)
#         v = v * token_mult[token_addr_inv[t]]
#         last_value[(t,m)] = v
#         metrics[m] = v
#     records.append({
#         "season": s, 
#         "token": t, 
#         **metrics 
#     })
# df_dep = pd.DataFrame(records)

df_dep['token_name'] = df_dep.token.apply(lambda v: token_addr_inv[v])

df_dep = df_dep.loc[df_dep.token_name == 'bean']
ddf(df_dep.tail(10))

,id,season,timestamp,totalDepositedAmount,totalDepositedBDV,token,token_name
1376,0xc1e088fc1323b20bcbee9bd1b9fc9546db5624c5-0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab-6554,6554,1661532658,1056188170996,1087416829247,0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab,bean
1377,0xc1e088fc1323b20bcbee9bd1b9fc9546db5624c5-0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab-6555,6555,1661535042,1062882630504,1100778041666,0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab,bean
1378,0xc1e088fc1323b20bcbee9bd1b9fc9546db5624c5-0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab-6556,6556,1661539883,1062128969177,1093353369134,0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab,bean
1382,0xc1e088fc1323b20bcbee9bd1b9fc9546db5624c5-0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab-6557,6557,1661543270,1062046840256,1093271240213,0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab,bean
1386,0xc1e088fc1323b20bcbee9bd1b9fc9546db5624c5-0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab-6558,6558,1661544829,1062066991894,1093307701463,0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab,bean
1387,0xc1e088fc1323b20bcbee9bd1b9fc9546db5624c5-0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab-6560,6560,1661549468,1060230534818,1091454934775,0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab,bean
1388,0xc1e088fc1323b20bcbee9bd1b9fc9546db5624c5-0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab-6559,6559,1661549468,1060230534818,1091454934775,0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab,bean
1389,0xc1e088fc1323b20bcbee9bd1b9fc9546db5624c5-0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab-6561,6561,1661555518,1060311998079,1091617861297,0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab,bean
1390,0xc1e088fc1323b20bcbee9bd1b9fc9546db5624c5-0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab-6562,6562,1661561368,1060312525373,1091537452624,0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab,bean
1391,0xc1e088fc1323b20bcbee9bd1b9fc9546db5624c5-0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab-6563,6563,1661563215,1060315493976,1091542862536,0xbea0000029ad1c77d3d5d23ba2d8893db9d1efab,bean


In [5]:
df_dep = df_dep.loc[df_dep.token_name == 'bean']
alt.Chart(df_dep).mark_line().encode(
    x=alt.X("season:O"), 
    y=alt.Y("totalDepositedAmount:Q"), 
    color=alt.Color("token_name:N"), 
    tooltip=alt.Tooltip("totalDepositedAmount:Q")
).properties(width=700)

alt.Chart(...)

In [6]:
"0xC1E088fC1323b20BCBee9bd1B9fC9546db5624C5".lower()

'0xc1e088fc1323b20bcbee9bd1b9fc9546db5624c5'

In [7]:
"0x1BEA0050E63e05FBb5D8BA2f10cf5800B6224449".lower()

'0x1bea0050e63e05fbb5d8ba2f10cf5800b6224449'

In [8]:
# query SeasonalDepositedUnripeBeans {
#   siloAssetDailySnapshots(
#     where: {
#       siloAsset: "0xc1e088fc1323b20bcbee9bd1b9fc9546db5624c5-0x1bea0050e63e05fbb5d8ba2f10cf5800b6224449"
#     }
#     orderBy: season
#     orderDirection: desc
#     first: 10 
#   ) {
#     season
#     siloAsset {
#       totalDepositedAmount
#     	totalDepositedBDV
#     }
#   }
# }

## Beanstalk Credit Breakdown (Daily + Cumulative)

Credit 
- Silo 
  - emissions (in-progress)
- Barn 
  - sprouts rinsed / rinsable (done)
- Field 
  - pods harvested (done)
  - pods harvestable (done)
  
Debt
- Field
  - pods issued (done) 
- Barn 
  - sprouts

In [9]:
@cache
def query_rewards(refresh=None): 
    rewards = bs.Query.rewards(orderBy="blockNumber", orderDirection="asc", first=10000)
    df_rewards = sg.query_df(
        [
            rewards.season, 
            rewards.toFertilizer, 
        ], 
        pagination_strategy=ShallowStrategy
    )
    return df_rewards 

In [111]:
# fertilizer emissions (incomplete season axis, all seasons unique)
df_rewards = query_rewards().copy()
df_rewards = remove_prefix(df_rewards, 'rewards_')
df_rewards['sprouts_rinsed_daily'] = df_rewards.toFertilizer.astype(float) / 1e6
df_rewards['sprouts_rinsed_cumulative'] = df_rewards.sprouts_rinsed_daily.cumsum()
df_rewards = df_rewards.drop(columns=["toFertilizer"])
assert all(v == 1 for v in df_rewards.season.value_counts().values)
df_rewards.tail()

,season,sprouts_rinsed_daily,sprouts_rinsed_cumulative
388,6559,68.851580,2.096716e+06
389,6560,17.212895,2.096733e+06
390,6561,68.851580,2.096802e+06
391,6562,86.064475,2.096888e+06
392,6563,86.064475,2.096974e+06


In [66]:
@cache 
def query_field(refresh=None) -> pd.DataFrame: 
    field_snaps = bs.Query.fieldDailySnapshots(
        orderBy="season", 
        orderDirection="asc", 
        first=10000, 
        where={"field": "0xc1e088fc1323b20bcbee9bd1b9fc9546db5624c5"}
    )
    df_field = sg.query_df(
        [
            field_snaps.season, 
            field_snaps.newHarvestedPods, 
            field_snaps.newHarvestablePods, 
            field_snaps.podIndex, 
        ], 
        pagination_strategy=ShallowStrategy
    )
    return df_field 

In [151]:
df_field = query_field(refresh=3).copy()
df_field = remove_prefix(df_field, "fieldDailySnapshots_")
df_field = df_field.sort_values("season")
df_field['pods_harvestable_daily'] = (df_field.newHarvestablePods / 10**6)
df_field['pods_harvested_daily'] = df_field.newHarvestedPods / 10**6
df_field = df_field.drop(columns=['newHarvestablePods', 'newHarvestedPods'])
df_field = df_field.groupby('season').agg({
    # handles edge case for season 6074 which occurred multiple times 
    "pods_harvestable_daily": "sum", 
    "pods_harvested_daily": "sum", 
    "podIndex": "max"
}).reset_index()
df_field['pods_issued_cumulative'] = df_field.podIndex / 10**6
df_field['pods_issued_daily'] = df_field.pods_issued_cumulative - df_field.pods_issued_cumulative.shift(1).fillna(0)
df_field['pods_harvested_cumulative'] = df_field.pods_harvested_daily.cumsum()
df_field = df_field.drop(columns=['podIndex'])
assert all(v == 1 for v in df_field.season.value_counts().values)
df_field.tail()

,season,pods_harvestable_daily,pods_harvested_daily,pods_issued_cumulative,pods_issued_daily,pods_harvested_cumulative
271,6489,1499.181633,0.000000,8.210963e+08,21662.109986,5.752809e+07
272,6513,3181.584188,5715.895364,8.210979e+08,1590.791450,5.753380e+07
273,6537,3188.827863,0.000000,8.210996e+08,1716.976045,5.753380e+07
274,6561,2881.202400,6671.011205,8.211010e+08,1440.600600,5.754048e+07
275,6564,325.812678,0.000000,8.211012e+08,162.906234,5.754048e+07


In [152]:
def silo_emissions_pre_replant() -> pd.DataFrame: 
    """Temporary solution to subgraph not having silo emissions pre-replant 
    
    Data was downloaded from dune 
    """
    with Path("data/SupplyIncrease.json").open('r') as f: 
        data = json.loads(f.read())
    data = [remove_keys(d['data'], ['__typename']) for d in data]
    df_supply_inc = pd.DataFrame(data)[['season', 'newSilo']]
    return df_supply_inc

@cache 
def query_silo(refresh=None) -> pd.DataFrame: 
    silo_snaps = bs.Query.siloDailySnapshots(
        orderBy="season", 
        orderDirection="asc", 
        first=10000, 
        where={"silo": "0xc1e088fc1323b20bcbee9bd1b9fc9546db5624c5"}
    )
    df = sg.query_df(
        [
            silo_snaps.season, 
            silo_snaps.dailyBeanMints, 
            # silo_snaps.totalBeanMints, # add back when subgraph includes historical data 
        ], 
        pagination_strategy=ShallowStrategy
    )
    return df 

In [153]:
# process post-replant silo data (subgraph)
df_silo = query_silo(refresh=1).copy()
df_silo = remove_prefix(df_silo, "siloDailySnapshots_")
df_silo = df_silo.loc[df_silo.season != 5903] # Subgraph bug probably? 
assert df_silo.season.min() == 6074, "If this fails, then subgraph was fixed to include historical data."
df_silo = df_silo.rename(columns={"dailyBeanMints": "silo_emissions_daily"})
# process pre-replant silo data (downloaded from dune)
df_silo_old = silo_emissions_pre_replant()
df_silo_old = df_silo_old.rename(columns={"newSilo": "silo_emissions_daily"})
# Combine pre and post replant data (no seasons in common so outer join)
df_silo = df_silo.merge(df_silo_old, how="outer")
assert set(df_silo.columns) == set(['season', 'silo_emissions_daily'])
df_silo = df_silo.sort_values("season")
df_silo = df_silo.groupby('season').agg({
    # handles edge case for season 6074 which occurred multiple times 
    "silo_emissions_daily": "sum", 
}).reset_index()
df_silo['silo_emissions_daily'] /= 10**6
df_silo['silo_emissions_cumulative'] = df_silo.silo_emissions_daily.cumsum()
assert all(v == 1 for v in df_silo.season.value_counts().values)
df_silo.head() 

,season,silo_emissions_daily,silo_emissions_cumulative
0,3,31.650670,31.650670
1,4,17.760260,49.410930
2,5,43.709604,93.120534
3,18,0.342173,93.462707
4,21,676.195254,769.657961


In [154]:
@cache 
def query_seasons(refresh=None) -> pd.DataFrame: 
    seasons = bs.Query.seasons(
        first=10000, orderBy="season", orderDirection="asc"
    )
    df = sg.query_df([
        seasons.season, 
        seasons.timestamp, 
    ], pagination_strategy=ShallowStrategy)
    df = remove_prefix(df, 'seasons_')
    return df 

In [155]:
df_szns = query_seasons()
df_szns['timestamp'] = pd.to_datetime(df_szns.timestamp, unit='s')
df_szns = df_szns.loc[df_szns.season >= 2] # timestamps are wrong for season 0 and 1 
assert all(v == 1 for v in df_szns.value_counts().values)
df_szns.head()

,season,timestamp
2,2,2021-08-07 00:06:08
3,3,2021-08-07 01:07:38
4,4,2021-08-07 02:09:28
5,5,2021-08-07 03:07:35
6,6,2021-08-07 04:11:23


In [158]:
alt.data_transformers.disable_max_rows()

df = df_szns.merge(
    df_rewards, how='left', on='season'
).merge(
    df_field, how='left', on='season'
).merge(
    df_silo, how='left', on='season'
)
assert len(df) == len(df_szns)
# df = df.loc[df.season >= 6075]
df = df.ffill().fillna(0) # Not technically correct but close enough 
# df_debt = df[[
#     'timestamp', 
#     'pods_issued_cumulative', 
# ]].melt(
#     id_vars=['timestamp'], 
#     value_vars=[
#         'pods_issued_cumulative', 
#     ]
# ).sort_values("timestamp").reset_index(drop=True)
# for c in df_debt.columns: 
#     if c != 'timestamp': 
#         df_debt[c] *= -1
df_credit = df[[
    'timestamp', 
    'sprouts_rinsed_cumulative', 
    'pods_harvestable_daily',
    'pods_harvested_cumulative', 
    'silo_emissions_cumulative',
]].melt(
    id_vars=['timestamp'], 
    value_vars=[
        'sprouts_rinsed_cumulative', 
        'pods_harvestable_daily',
        'pods_harvested_cumulative', 
        'silo_emissions_cumulative',
    ]
).sort_values("timestamp").reset_index(drop=True)
# df_credit = df_credit.loc[df_credit.variable == 'silo_emissions_cumulative']
credit = alt.Chart(df_credit).mark_area().encode(
    x="yearmonthdatehoursminutes(timestamp):T", 
    y="value:Q", 
    color="variable:N"
).properties(width=750, title="Beanstalk Credit")
# debt = alt.Chart(df_debt).mark_area().encode(
#     x="yearmonthdatehoursminutes(timestamp):T", 
#     y="value:Q", 
#     color="variable:N"
# ).properties(width=750)
# credit & debt
credit

alt.Chart(...)

In [159]:
df.tail()

,season,timestamp,sprouts_rinsed_daily,sprouts_rinsed_cumulative,pods_harvestable_daily,pods_harvested_daily,pods_issued_cumulative,pods_issued_daily,pods_harvested_cumulative,silo_emissions_daily,silo_emissions_cumulative
6558,6560,2022-08-26 22:00:56,17.212895,2.096733e+06,3188.827863,0.000000,8.210996e+08,1716.976045,5.753380e+07,3188.827871,7.623241e+07
6559,6561,2022-08-26 23:00:18,68.851580,2.096802e+06,2881.202400,6671.011205,8.211010e+08,1440.600600,5.754048e+07,2881.202409,7.623529e+07
6560,6562,2022-08-27 00:00:06,86.064475,2.096888e+06,2881.202400,6671.011205,8.211010e+08,1440.600600,5.754048e+07,2881.202409,7.623529e+07
6561,6563,2022-08-27 01:00:03,86.064475,2.096974e+06,2881.202400,6671.011205,8.211010e+08,1440.600600,5.754048e+07,2881.202409,7.623529e+07
6562,6564,2022-08-27 02:00:25,86.064475,2.096974e+06,325.812678,0.000000,8.211012e+08,162.906234,5.754048e+07,325.812680,7.623561e+07
